In [ ]:

# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

In [ ]:

# Part 1 - Data Preprocessing

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values


In [3]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/home/fellipe/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/fellipe/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Part 2 - Now let's make the ANN!

In [6]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [7]:
# Initialising the ANN
classifier = Sequential()

In [8]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
# classifier.add(Dropout(rate = 0.1))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dropout(rate = 0.1))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
8000/8000 [==============================] - 1s 101us/step - loss: 0.4777 - acc: 0.7960
Epoch 2/10
8000/8000 [==============================] - 1s 85us/step - loss: 0.4232 - acc: 0.7960
Epoch 3/10
8000/8000 [==============================] - 1s 90us/step - loss: 0.4195 - acc: 0.8109
Epoch 4/10
8000/8000 [==============================] - 1s 82us/step - loss: 0.4172 - acc: 0.8269
Epoch 5/10
8000/8000 [==============================] - 1s 83us/step - loss: 0.4148 - acc: 0.8282
Epoch 6/10
8000/8000 [==============================] - 1s 86us/step - loss: 0.4129 - acc: 0.8294
Epoch 7/10
8000/8000 [==============================] - 1s 98us/step - loss: 0.4114 - acc: 0.8319
Epoch 8/10
8000/8000 [==============================] - 1s 155us/step - loss: 0.4100 - acc: 0.8332
Epoch 9/10
8000/8000 [==============================] - 1s 124us/step - loss: 0.4083 - acc: 0.

In [ ]:
# Part 3 - Making predictions and evaluating the model

In [9]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [10]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

In [11]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [12]:
print(cm)

[[1550   45]
 [ 267  138]]


In [ ]:
# Part 4 - Evaluating, Improving and Tuning the ANN

In [13]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import Dropout
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [14]:
print(accuracies)
print(mean)
print(variance)

[0.78625    0.79       0.82375    0.81875    0.84874999 0.82625
 0.82249999 0.81249999 0.81875    0.845     ]
0.8192499963939192
0.01898354417489961


In [ ]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [15]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

In [16]:

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [17]:
classifier = KerasClassifier(build_fn = build_classifier)

In [18]:
parameters = {'batch_size': [5, 10],
              'epochs': [5, 10],
              'optimizer': ['adam', 'rmsprop']}

In [19]:

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [20]:
grid_search = grid_search.fit(X_train, y_train)

Epoch 9/10
7200/7200 [==============================] - 2s 304us/step - loss: 0.4370 - acc: 0.7971
Epoch 10/10
7200/7200 [==============================] - 2s 300us/step - loss: 0.4323 - acc: 0.7971
Epoch 1/10
7200/7200 [==============================] - 6s 886us/step - loss: 0.5034 - acc: 0.7964
Epoch 2/10
7200/7200 [==============================] - 2s 247us/step - loss: 0.4362 - acc: 0.7967
Epoch 3/10
7200/7200 [==============================] - 2s 272us/step - loss: 0.4333 - acc: 0.7967
Epoch 4/10
7200/7200 [==============================] - 2s 256us/step - loss: 0.4280 - acc: 0.7967
Epoch 5/10
7200/7200 [==============================] - 2s 242us/step - loss: 0.4249 - acc: 0.7967
Epoch 6/10
7200/7200 [==============================] - 2s 251us/step - loss: 0.4291 - acc: 0.7967
Epoch 7/10
7200/7200 [==============================] - 2s 250us/step - loss: 0.4264 - acc: 0.8067
Epoch 8/10
7200/7200 [==============================] - 2s 243us/step - loss: 0.4273 - acc: 0.8218
Epoch 9/1

In [21]:

best_parameters = grid_search.best_params_

In [22]:

best_accuracy = grid_search.best_score_

In [23]:

print(best_parameters)

{'batch_size': 5, 'epochs': 10, 'optimizer': 'rmsprop'}


In [24]:
print(best_accuracy)

0.832375
